In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join("../helpers"))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import optuna
import functools

import sklearn.linear_model as sklm
import sklearn.model_selection as skms
import sklearn.metrics as metrics
import sklearn.pipeline as skpl
import sklearn.preprocessing as skpp
import sklearn.compose as skcmp
import sklearn.impute as skim
import category_encoders as ce

import lightgbm as lgb
import xgboost as xgb

from preprocessing import *
from ucimlrepo import fetch_ucirepo

import gc

### Load and process data

In [2]:
train_df = pd.read_csv("../data/train.csv", index_col="id")
test_df = pd.read_csv("../data/test.csv", index_col="id")
orig_df = pd.read_csv("../data/orig.csv", index_col="id")

CONT_FEATS = [
    "cap-diameter",
    "stem-height",
    "stem-width"
]
CAT_FEATS = [c for c in train_df.columns if c not in CONT_FEATS]
RESPONSE_COL = "class"

In [3]:
fix_categories(orig_df, train_df, test_df, CAT_FEATS, nan_as_cat=True)
fix_that_one_mushroom_in_test(test_df)

pipeline = skpl.make_pipeline(
    skcmp.ColumnTransformer([
        (
            "float", 
            skpl.make_pipeline
            (
                skim.SimpleImputer(add_indicator=True, strategy="median"),
                skpp.StandardScaler().set_output(transform="pandas") # This is also scaling the indicator column...unsure if it matters
            ),
            CONT_FEATS
        ),
        (
            "cat", 
            skpp.OneHotEncoder(
                sparse_output=False,
                dtype=np.int8,
                handle_unknown='infrequent_if_exist'),
            [c for c in CAT_FEATS if c != RESPONSE_COL]
        ),
    ], remainder="passthrough").set_output(transform="pandas"),
    )

d:\Programming\Workspaces\Python3 Workspace\Misc Data Analysis\Kaggle Playground Competition\S4E8\helpers\preprocessing.py:54: UserWarning: 'class' not found in dataframe. Probably the response column in the test dataframe.
  warnings.warn(


### Helper methods for creating the models to be fed into optuna

In [4]:
def make_xgb(**kwargs):
    return xgb.XGBClassifier(
        enable_categorical=True,
        random_state=0,
        n_jobs=-1,
        **kwargs
    )

def optuna_xgb(trial):
    return make_xgb(
        n_estimators=trial.suggest_int("n_estimators", 1, 1e4),
        eta=trial.suggest_float("eta", 0.01, 1),
        gamma=trial.suggest_float("gamma", 0, 20),
        max_depth=trial.suggest_int("max_depth", 3, 50),
        max_leaves=trial.suggest_int("max_leaves", 0, 1e4),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.05, 1),
        colsample_bylevel=trial.suggest_float("colsample_bylevel", 0.05, 1),
        colsample_bynode=trial.suggest_float("colsample_bynode", 0.05, 1),
        reg_lambda=trial.suggest_int("reg_lambda", 0, 20),
        reg_alpha=trial.suggest_int("reg_alpha", 0, 20),
        grow_policy=trial.suggest_categorical(
            "grow_policy", ["depthwise", "lossguide"]
        ),
        min_child_weight=trial.suggest_float("min_child_weight", 0, 1e3),
        subsample=trial.suggest_float("subsample", 0.01, 1),
        max_delta_step=trial.suggest_float("max_delta_step", 0, 1e2)
    )

def make_lgbm(**kwargs):
    return lgb.LGBMClassifier(
        boosting_type="gbdt",
        n_jobs=-1,
        **kwargs
    )

def optuna_lgbm(trial):
    return make_lgbm(
        num_leaves=trial.suggest_int("num_leaves", 15, 45),
        max_depth=trial.suggest_int("max_depth", -1, 50),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1),
        n_estimators=trial.suggest_int("n_estimators", 1, 1e4),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.05, 1),
        reg_lambda=trial.suggest_int("reg_lambda", 0, 20),
        reg_alpha=trial.suggest_int("reg_alpha", 0, 20),
    )

def make_dart(**kwargs):
    return lgb.LGBMClassifier(
        boosting_type="dart",
        n_jobs=-1,
        **kwargs
    )

def optuna_dart(trial):
    return make_dart(
        num_leaves=trial.suggest_int("num_leaves", 15, 45),
        max_depth=trial.suggest_int("max_depth", -1, 50),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1),
        n_estimators=trial.suggest_int("n_estimators", 1, 500),
        colsample_bytree=trial.suggest_float("colsample_bytree", 0.05, 1),
        reg_lambda=trial.suggest_int("reg_lambda", 0, 20),
        reg_alpha=trial.suggest_int("reg_alpha", 0, 20),
    )

In [5]:
def objective_xgb(trial, tx, ty, vx, vy):
    model = optuna_xgb(trial)
    model.fit(tx, ty)
    preds = model.predict(vx)

    return metrics.matthews_corrcoef(vy, preds)

def objective_lgbm(trial, tx, ty, vx, vy):
    model = optuna_lgbm(trial)
    model.fit(tx, ty, eval_metric="mcc", eval_set=(vx, vy), callbacks=[lgb.early_stopping(stopping_rounds=5)])
    preds = model.predict(vx)
    return metrics.matthews_corrcoef(vy, preds)

def objective_dart(trial, tx, ty, vx, vy):
    model = optuna_dart(trial)
    model.fit(tx, ty, eval_metric="mcc")
    preds = model.predict(vx)
    return metrics.matthews_corrcoef(vy, preds)

In [6]:
def refit_save_predictions(model, train_x, train_y, test_x, path, file_prefix):
    root_dir = os.path.abspath(path)
    train_file = os.path.join(root_dir, f"{file_prefix}_train.csv")
    test_file = os.path.join(root_dir, f"{file_prefix}_test.csv")

    model.fit(train_x, train_y)

    train_preds = model.predict(train_x)
    train_pred_proba = model.predict_proba(train_x)

    test_preds = model.predict(test_x)
    test_pred_proba = model.predict_proba(test_x)

    out_df = pd.DataFrame({RESPONSE_COL:train_preds}, index=train_x.index)
    for i in range(train_pred_proba.shape[1]):
        out_df[f"pp_{i}"] = train_pred_proba[:, i]
    out_df.to_csv(train_file)

    out_df = pd.DataFrame({RESPONSE_COL:test_preds}, index=test_x.index)
    for i in range(test_pred_proba.shape[1]):
        out_df[f"pp_{i}"] = test_pred_proba[:, i]
    out_df.to_csv(test_file)

### Model training

In [7]:
x_train = pipeline.fit_transform(train_df.drop(columns=[RESPONSE_COL]))
y_train = train_df[RESPONSE_COL]
y_train = y_train.replace({"e":0, "p":1})

x_test = pipeline.transform(test_df)

x_tr, x_v, y_tr, y_v = skms.train_test_split(
    x_train, 
    y_train, 
    stratify=y_train
)

#### XGB

In [8]:
# optuna.delete_study(study_name="xgb_tuning_nan_cat", storage="sqlite:///optuna.sqlite3")

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///optuna.sqlite3",
    study_name="xgb_tuning_nan_cat",
    load_if_exists=True
)

obj = functools.partial(objective_xgb, tx=x_tr, ty=y_tr, vx=x_v, vy=y_v)
study.optimize(obj, n_trials=1)
gc.collect()

[I 2024-08-16 10:34:49,718] Using an existing study with name 'xgb_tuning_nan_cat' instead of creating a new one.
[I 2024-08-16 10:43:01,141] Trial 51 finished with value: 0.9839299090349225 and parameters: {'n_estimators': 3937, 'eta': 0.05957287291848978, 'gamma': 0.8345920072739275, 'max_depth': 14, 'max_leaves': 4275, 'colsample_bytree': 0.8102869845591929, 'colsample_bylevel': 0.8965487399448485, 'colsample_bynode': 0.6181478505746691, 'reg_lambda': 12, 'reg_alpha': 1, 'grow_policy': 'depthwise', 'min_child_weight': 395.6856055888533, 'subsample': 0.8922052974031557, 'max_delta_step': 63.36878846075586}. Best is trial 50 with value: 0.9843358600724808.


2930

In [9]:
study = optuna.load_study(study_name="xgb_tuning_nan_cat", storage="sqlite:///optuna.sqlite3")
model = make_xgb(**study.best_params)
refit_save_predictions(model, x_train, y_train, x_test, "../predictions/v2", "xgb")

#### LGBM

In [10]:
# optuna.delete_study(study_name="lgbm_tuning_nan_cat", storage="sqlite:///optuna.sqlite3")

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///optuna.sqlite3",
    study_name="lgbm_tuning_nan_cat",
    load_if_exists=True
)

obj = functools.partial(objective_lgbm, tx=x_tr, ty=y_tr, vx=x_v, vy=y_v)
study.optimize(obj, n_trials=1)
gc.collect()

[I 2024-08-16 11:01:34,403] Using an existing study with name 'lgbm_tuning_nan_cat' instead of creating a new one.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.118004 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[871]	valid_0's binary_logloss: 0.0366412


[I 2024-08-16 11:02:28,699] Trial 57 finished with value: 0.9843347982816744 and parameters: {'num_leaves': 35, 'max_depth': 46, 'learning_rate': 0.08898471672808295, 'n_estimators': 2849, 'colsample_bytree': 0.2855155587749621, 'reg_lambda': 8, 'reg_alpha': 1}. Best is trial 57 with value: 0.9843347982816744.


6794

In [11]:
study = optuna.load_study(study_name="lgbm_tuning_nan_cat", storage="sqlite:///optuna.sqlite3")
model = make_lgbm(**study.best_params)
refit_save_predictions(model, x_train, y_train, x_test, "../predictions/v2", "lgbm")

[LightGBM] [Info] Number of positive: 1705396, number of negative: 1411549
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.145431 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 3116945, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


#### Dart

In [12]:
# optuna.delete_study(study_name="dart_tuning_nan_cat", storage="sqlite:///optuna.sqlite3")

study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///optuna.sqlite3",
    study_name="dart_tuning_nan_cat",
    load_if_exists=True
)

obj = functools.partial(objective_dart, tx=x_tr, ty=y_tr, vx=x_v, vy=y_v)
study.optimize(obj, n_trials=50)
gc.collect()

[I 2024-08-16 11:08:25,029] A new study created in RDB with name: dart_tuning_nan_cat


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080582 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2024-08-16 11:08:49,696] Trial 0 finished with value: 0.9063121082471411 and parameters: {'num_leaves': 22, 'max_depth': 34, 'learning_rate': 0.06255115741113876, 'n_estimators': 129, 'colsample_bytree': 0.10744878781062826, 'reg_lambda': 12, 'reg_alpha': 16}. Best is trial 0 with value: 0.9063121082471411.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087064 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

[I 2024-08-16 11:11:51,997] Trial 1 finished with value: 0.9557578761046263 and parameters: {'num_leaves': 45, 'max_depth': 27, 'learning_rate': 0.027250464572051627, 'n_estimators': 382, 'colsample_bytree': 0.13296948700495573, 'reg_lambda': 5, 'reg_alpha': 20}. Best is trial 1 with value: 0.9557578761046263.


...

[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098141 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-16 13:23:46,947] Trial 48 finished with value: 0.9804686524852635 and parameters: {'num_leaves': 38, 'max_depth': 16, 'learning_rate': 0.07771744613521209, 'n_estimators': 416, 'colsample_bytree': 0.42842936275090526, 'reg_lambda': 7, 'reg_alpha': 4}. Best is trial 5 with value: 0.9822310490038294.


[LightGBM] [Info] Number of positive: 1279047, number of negative: 1058661
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.112118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 2337708, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110


[I 2024-08-16 13:26:56,790] Trial 49 finished with value: 0.975078417197354 and parameters: {'num_leaves': 21, 'max_depth': 21, 'learning_rate': 0.08188529711264368, 'n_estimators': 463, 'colsample_bytree': 0.36522077245398255, 'reg_lambda': 10, 'reg_alpha': 6}. Best is trial 5 with value: 0.9822310490038294.


9848

In [14]:
study = optuna.load_study(study_name="dart_tuning_nan_cat", storage="sqlite:///optuna.sqlite3")
model = make_dart(**study.best_params)
refit_save_predictions(model, x_train, y_train, x_test, "../predictions/v2", "dart")

[LightGBM] [Info] Number of positive: 1705396, number of negative: 1411549
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.125574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1029
[LightGBM] [Info] Number of data points in the train set: 3116945, number of used features: 135
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547137 -> initscore=0.189110
[LightGBM] [Info] Start training from score 0.189110
